In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTEENN
from sklearn.ensemble import ExtraTreesClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import balanced_accuracy_score

In [3]:
filename ='data/star_classification.csv'
sdss17_df_raw = pd.read_csv(filename)

num_rows = np.shape(sdss17_df_raw)[0]
num_cols = np.shape(sdss17_df_raw)[1]

sdss17_df = sdss17_df_raw.drop(['obj_ID', 'alpha', 'delta', 'run_ID', 'rerun_ID', 'cam_col', 'field_ID', 'fiber_ID', 'spec_obj_ID', 'MJD', 'plate'], axis = 1)

# Remove Very Bright Objects
sdss17_df_no_anomaly = sdss17_df[sdss17_df['u'] > -1]

num_rows = np.shape(sdss17_df_no_anomaly)[0]
num_cols = np.shape(sdss17_df_no_anomaly)[1]

sdss17_features = sdss17_df_no_anomaly.drop('class', axis=1, inplace=False)
sdss17_class_categorical = sdss17_df_no_anomaly['class']
classes = sdss17_class_categorical.unique() # redundant - should be created earlier

X_train, X_test, y_train, y_test = train_test_split(
    sdss17_features,
    sdss17_class_categorical,
    test_size=0.33,
    random_state=1000)

In [12]:
scorers = {
            'f1_score': make_scorer(f1_score, average='macro'),
            'precision_score': make_scorer(precision_score, average='macro'),
            'recall_score': make_scorer(recall_score, average='macro'),
            'accuracy_score': make_scorer(accuracy_score)
          }

def multi_gridsearch(pipes, parameters):
    """
    Apply Grid Search to different pipelines of the same model.
    
    pipes: a list Pipeline objects
    parameters: the parameter grid for Grid Search
    """
    for pipe in pipes:
        print(pipe)
        grid_pipeline = GridSearchCV(pipe, parameters, verbose=10, cv=3, scoring=scorers, refit="precision_score")
        grid_pipeline.fit(X_train,y_train)
        print(grid_pipeline.best_params_, end='\n\n')

In [5]:

smote = Pipeline([('SMOTE', SMOTE(random_state=0)),
                  ('rfc', RandomForestClassifier(random_state=0))
                 ])

roversample = Pipeline([('Oversample', RandomOverSampler(random_state=0)),
                        ('rfc', RandomForestClassifier(random_state=0))
                       ])



smoteenn = Pipeline([('SMOTEENN', SMOTEENN(random_state=0)),
                  ('rfc', RandomForestClassifier(random_state=0))
                 ])

param_grid = {
  'rfc__n_estimators':[500, 1000],
  'rfc__min_samples_split': [2, 4]
  }

pipeline_list= [smote, roversample, smoteenn]
multi_gridsearch(pipeline_list, param_grid)

Pipeline(steps=[('SMOTE', SMOTE()), ('rfc', RandomForestClassifier())])
Fitting 3 folds for each of 4 candidates, totalling 12 fits
{'rfc__min_samples_split': 4, 'rfc__n_estimators': 500}

Pipeline(steps=[('Oversample', RandomOverSampler()),
                ('rfc', RandomForestClassifier())])
Fitting 3 folds for each of 4 candidates, totalling 12 fits
{'rfc__min_samples_split': 2, 'rfc__n_estimators': 1000}

Pipeline(steps=[('SMOTEENN', SMOTEENN()), ('rfc', RandomForestClassifier())])
Fitting 3 folds for each of 4 candidates, totalling 12 fits
{'rfc__min_samples_split': 4, 'rfc__n_estimators': 1000}



In [6]:

smote = Pipeline([('SMOTE', SMOTE(random_state=0)),
                  ('et', ExtraTreesClassifier(random_state=0))
                 ])

roversample = Pipeline([('Oversample', RandomOverSampler(random_state=0)),
                        ('et', ExtraTreesClassifier(random_state=0))
                       ])



smoteenn = Pipeline([('SMOTEENN', SMOTE(random_state=0)),
                  ('et', ExtraTreesClassifier(random_state=0))
                 ])



param_grid = {
  'et__n_estimators':[500, 1000],
  'et__min_samples_split': [2, 4]
  }

pipeline_list=[smote, roversample, smoteenn]

multi_gridsearch(pipeline_list, param_grid)

Pipeline(steps=[('SMOTE', SMOTE()), ('et', ExtraTreesClassifier())])
Fitting 3 folds for each of 4 candidates, totalling 12 fits
{'et__min_samples_split': 4, 'et__n_estimators': 500}

Pipeline(steps=[('Oversample', RandomOverSampler()),
                ('et', ExtraTreesClassifier())])
Fitting 3 folds for each of 4 candidates, totalling 12 fits
{'et__min_samples_split': 2, 'et__n_estimators': 1000}

Pipeline(steps=[('SMOTEENN', SMOTE()), ('et', ExtraTreesClassifier())])
Fitting 3 folds for each of 4 candidates, totalling 12 fits
{'et__min_samples_split': 2, 'et__n_estimators': 500}



In [11]:
brf = Pipeline([('brf', BalancedRandomForestClassifier())])


param_grid = {
  'brf__n_estimators':[500, 1000],
  'brf__min_samples_split': [2, 4]
  }

pipeline_list=[brf]

multi_gridsearch(pipeline_list, param_grid)

Pipeline(steps=[('brf', BalancedRandomForestClassifier())])
Fitting 3 folds for each of 4 candidates, totalling 12 fits
{'brf__min_samples_split': 2, 'brf__n_estimators': 1000}



In [13]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
KNN = Pipeline([('Scale', StandardScaler()),
                ('PCA', PCA()),
                ('SMOTEENN', SMOTEENN()),
                ('KNN', KNeighborsClassifier())
])

parameters_KNN = {
    'KNN__n_neighbors': [3, 5, 10],
    'KNN__leaf_size': [10, 30, 40],
    'KNN__p': [1,2],
    'KNN__weights': ['uniform', 'distance'],
    'KNN__metric': ['minkowski', 'chebyshev']
}

pipeline_list = [KNN]

multi_gridsearch(pipeline_list, parameters_KNN)




Pipeline(steps=[('Scale', StandardScaler()), ('PCA', PCA()),
                ('SMOTEENN', SMOTEENN()), ('KNN', KNeighborsClassifier())])
Fitting 3 folds for each of 72 candidates, totalling 216 fits
[CV 1/3; 1/72] START KNN__leaf_size=10, KNN__metric=minkowski, KNN__n_neighbors=3, KNN__p=1, KNN__weights=uniform


/home/fabio/.venv/myproj/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


[CV 1/3; 1/72] END KNN__leaf_size=10, KNN__metric=minkowski, KNN__n_neighbors=3, KNN__p=1, KNN__weights=uniform; accuracy_score: (test=0.949) f1_score: (test=0.944) precision_score: (test=0.932) recall_score: (test=0.957) total time=   3.0s
[CV 2/3; 1/72] START KNN__leaf_size=10, KNN__metric=minkowski, KNN__n_neighbors=3, KNN__p=1, KNN__weights=uniform


/home/fabio/.venv/myproj/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


[CV 2/3; 1/72] END KNN__leaf_size=10, KNN__metric=minkowski, KNN__n_neighbors=3, KNN__p=1, KNN__weights=uniform; accuracy_score: (test=0.948) f1_score: (test=0.943) precision_score: (test=0.930) recall_score: (test=0.958) total time=   3.2s
[CV 3/3; 1/72] START KNN__leaf_size=10, KNN__metric=minkowski, KNN__n_neighbors=3, KNN__p=1, KNN__weights=uniform


...

[CV 1/3; 71/72] END KNN__leaf_size=40, KNN__metric=chebyshev, KNN__n_neighbors=10, KNN__p=2, KNN__weights=uniform; accuracy_score: (test=0.947) f1_score: (test=0.942) precision_score: (test=0.931) recall_score: (test=0.954) total time=   2.3s
[CV 2/3; 71/72] START KNN__leaf_size=40, KNN__metric=chebyshev, KNN__n_neighbors=10, KNN__p=2, KNN__weights=uniform


/home/fabio/.venv/myproj/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


[CV 2/3; 71/72] END KNN__leaf_size=40, KNN__metric=chebyshev, KNN__n_neighbors=10, KNN__p=2, KNN__weights=uniform; accuracy_score: (test=0.945) f1_score: (test=0.940) precision_score: (test=0.927) recall_score: (test=0.955) total time=   2.2s
[CV 3/3; 71/72] START KNN__leaf_size=40, KNN__metric=chebyshev, KNN__n_neighbors=10, KNN__p=2, KNN__weights=uniform


/home/fabio/.venv/myproj/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


[CV 3/3; 71/72] END KNN__leaf_size=40, KNN__metric=chebyshev, KNN__n_neighbors=10, KNN__p=2, KNN__weights=uniform; accuracy_score: (test=0.946) f1_score: (test=0.940) precision_score: (test=0.929) recall_score: (test=0.954) total time=   3.1s
[CV 1/3; 72/72] START KNN__leaf_size=40, KNN__metric=chebyshev, KNN__n_neighbors=10, KNN__p=2, KNN__weights=distance
[CV 1/3; 72/72] END KNN__leaf_size=40, KNN__metric=chebyshev, KNN__n_neighbors=10, KNN__p=2, KNN__weights=distance; accuracy_score: (test=0.945) f1_score: (test=0.940) precision_score: (test=0.928) recall_score: (test=0.954) total time=   2.3s
[CV 2/3; 72/72] START KNN__leaf_size=40, KNN__metric=chebyshev, KNN__n_neighbors=10, KNN__p=2, KNN__weights=distance
[CV 2/3; 72/72] END KNN__leaf_size=40, KNN__metric=chebyshev, KNN__n_neighbors=10, KNN__p=2, KNN__weights=distance; accuracy_score: (test=0.942) f1_score: (test=0.937) precision_score: (test=0.923) recall_score: (test=0.954) total time=   1.9s
[CV 3/3; 72/72] START KNN__leaf_siz